### 回帰モデル　改訂版

### 必要ライブラリの導入

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib -q

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.2f}'.format

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

### データ読み込み

#### データを読み込み

In [ ]:
from sklearn.datasets import fetch_california_housing

# カリフォルニアデータセットをロード
data = fetch_california_housing(as_frame=True)

# 特徴量とターゲットをデータフレームに変換
features = data.data
target = data.target

# 特徴量とターゲットを一つのデータフレームに結合
df = pd.concat([features, target.rename("MedianHouseValue")], axis=1)

# データフレームを表示
display(df.head())

#### データセットの説明文を表示

In [ ]:
print(data.DESCR)

### データ加工

#### データ絞り込み(項目MedIncのみ)
MedIncはMedium Income (収入の中央値)

In [ ]:
x_data = df[['MedInc']].values
print('絞り込み後')
print(f'shape: {x_data.shape}')
print(f'先頭5行:\n{x_data[:5]}')

#### ダミー変数を追加

In [ ]:
x = np.insert(x_data, 0, 1.0, axis=1)
print('ダミー変数追加後')
print(f'shape: {x.shape}')
print(f'先頭5行:\n{x[:5]}')

#### 正解データytの設定

In [ ]:
yt = df['MedianHouseValue'].values
print('正解データ')
print(f'shape: {yt.shape}')
print(f'先頭5行:\n{yt[:5]}')


#### 散布図表示

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 散布図表示
plt.scatter(x[:,1], yt, s=10, c='b')

# グラフのキレイ化
plt.title('収入 vs 不動産価格の散布図', fontsize=14)
plt.grid()
plt.xlabel('収入',fontsize=14)
plt.ylabel('不動産価格',fontsize=14)
plt.show()

### 関数定義

#### 予測関数

In [ ]:
# 予測関数 (1, x)の値から予測値ypを計算する
def pred(x, w):
    return(x @ w)

### 単回帰モデル

#### 初期設定

In [ ]:
# 初期化処理

# データ系列総数
M = x.shape[0]

# 入力データ次元数(ダミー変数を含む)
D = x.shape[1]

# 繰り返し回数
iters = 5000

# 学習率
alpha = 0.005

# 重みベクトルの初期値 (すべての値を1にする)
w = np.ones(D)

# 評価結果記録用 (損失関数値のみ記録)
history = np.zeros((0,2))

 #### 繰り返し処理

In [ ]:
# 繰り返しループ
for k in range(iters):

    # 予測値の計算 (7.8.1)
    yp = pred(x, w)

    # 誤差の計算 (7.8.2)
    yd = yp - yt

    # 勾配降下法の実装 (7.8.4)
    w = w - alpha * (x.T @ yd) / M

    # 学習曲線描画用データの計算、保存
    if ( k % 100 == 0):
        # 損失関数値の計算 (7.6.1)
        loss = np.mean(yd ** 2) / 2
        # 計算結果の記録
        history = np.vstack((history, np.array([k, loss])))
        # 画面表示
        print( "iter = %d  loss = %f" % (k, loss))

### 結果分析

#### 損失確認

In [ ]:
# 最終的な損失関数初期値、最終値
print(f'損失関数初期値:{history[0,1]:.04f}')
print(f'損失関数最終値:{history[-1,1]:.04f}')

#### 学習曲線表示

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,1])

# グラフのキレイ化
plt.title('学習曲線(損失)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 回帰直線表示

In [ ]:
# 下記直線描画用の座標値計算
xall = x[:,1].ravel()
xl = np.array([[1, xall.min()],[1, xall.max()]])
yl = pred(xl, w)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 散布図表示
plt.scatter(x[:,1], yt, s=10, c='b')

# 回帰直線表示
plt.plot(xl[:,1], yl, c='k')

# グラフのキレイ化
plt.title('散布図と回帰直線', fontsize=14)
plt.grid()
plt.xlabel('収入', fontsize=14)
plt.ylabel('不動産価格', fontsize=14)
plt.show()

### 重回帰モデル

#### 学習データ加工

In [ ]:
# 列(AveRooms: 平均部屋数)の追加
x_add = df[['AveRooms']].values
x2 = np.hstack((x, x_add))

print('変数追加後')
print(f'shape: {x2.shape}')
print(f'先頭5行:\n{x2[:5]}')

#### 初期設定

In [ ]:
# 初期化処理

# データ系列総数
M  = x2.shape[0]

# 入力データ次元数(ダミー変数を含む)
D = x2.shape[1]

# 繰り返し回数
iters = 5000

# 学習率
alpha = 0.005

# 重みベクトルの初期値 (すべての値を1にする)
w = np.ones(D)

# 評価結果記録用 (損失関数値のみ記録)
history = np.zeros((0,2))

#### 繰り返し処理

In [ ]:
# 繰り返しループ
for k in range(iters):

    # 予測値の計算 (7.8.1)
    yp = pred(x2, w)

    # 誤差の計算 (7.8.2)
    yd = yp - yt

    # 勾配降下法の実装 (7.8.4)
    w = w - alpha * (x2.T @ yd) / M

    # 学習曲線描画用データの計算、保存
    if ( k % 100 == 0):
        # 損失関数値の計算 (7.6.1)
        loss = np.mean(yd ** 2) / 2
        # 計算結果の記録
        history = np.vstack((history, np.array([k, loss])))
        # 画面表示
        print( "iter = %d  loss = %f" % (k, loss))

###  結果確認

#### 損失確認

In [ ]:
# 最終的な損失関数初期値、最終値
print(f'損失関数初期値:{history[0,1]:.04f}')
print(f'損失関数最終値:{history[-1,1]:.04f}')

#### 学習曲線表示

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示
plt.plot(history[1:,0], history[1:,1])

# グラフのキレイ化
plt.title('学習曲線(損失)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions